In [1]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.2.7 requires tyro, which is not installed.
unsloth-zoo 2025.2.7 requires protobuf<4.0.0, b

In [2]:
import torch
import gdown
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextStreamer
from trl import SFTTrainer
from sklearn.model_selection import train_test_split

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [7]:
file_path = "/content/poem_lucbat_realfinal_dataset.csv"
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,title,content,source,url
0,0,Việt Bắc,Tiếng ai tha thiết bên cồn\nBâng khuâng trong ...,10-1954\n\nChiến dịch Điện Biên Phủ kết thúc t...,https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu...
1,1,Khi con tu hú,"Khi con tu hú gọi bầy\nLúa chiêm đang chín, tr...","Huế, tháng 7-1939\n\nBài thơ Khi con tu hú đượ...",https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu...
2,2,Tiếng ru,"Con ong làm mật, yêu hoa\nCon cá bơi, yêu nước...",Ba khổ thơ đầu bài này được sử dụng trong SGK ...,https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu...
3,3,Lịch sử nước ta,"Dân ta phải biết sử ta,\nCho tường gốc tích nư...",Đầu năm 1942\n\nĐể giáo dục tinh thần yêu nước...,https://www.thivien.net/H%E1%BB%93-Ch%C3%AD-Mi...
4,4,Lỡ bước sang ngang,"“- Em ơi, em ở lại nhà,\nVườn dâu em đốn, mẹ g...",Bài thơ này được đăng lần đầu trên Tiểu thuyết...,https://www.thivien.net/Nguy%E1%BB%85n-B%C3%AD...


In [8]:
train_df, valid_df = train_test_split(df, test_size=0.1, random_state=42)

dataset = {
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(valid_df),
}

print(f"Training size: {len(dataset['train'])}, Validation size: {len(dataset['validation'])}")

Training size: 90, Validation size: 10


In [9]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

In [11]:
import textwrap

def format_poem(example):
    theme = example["title"]
    content = example["content"].strip()

    text = f"""<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Bạn là một AI có khả năng sáng tác thơ lục bát, và phải tuân thủ nghiêm ngặt quy tắc của thơ lục bát:
- Bắt đầu với dòng thơ 6 chữ, tiếp theo là dòng 8 chữ và lặp lại.
- Dòng thơ cuối cùng phải kết thúc bằng dòng 8 chữ.

<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Hãy sáng tác một bài thơ lục bát về chủ đề '{theme}'.

<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
{content}
<|eot_id|>"""

    return {"text": text}

dataset["train"] = dataset["train"].map(format_poem, remove_columns=dataset["train"].column_names)
dataset["validation"] = dataset["validation"].map(format_poem, remove_columns=dataset["validation"].column_names)

print(dataset["train"][0]["text"])

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Bạn là một AI có khả năng sáng tác thơ lục bát, và phải tuân thủ nghiêm ngặt quy tắc của thơ lục bát:
- Bắt đầu với dòng thơ 6 chữ, tiếp theo là dòng 8 chữ và lặp lại.
- Dòng thơ cuối cùng phải kết thúc bằng dòng 8 chữ.

<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Hãy sáng tác một bài thơ lục bát về chủ đề 'Quê tôi'.

<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Quê tôi có gió bốn mùa
Có giăng giữa tháng, có chùa quanh năm.
Chuông hôm, gió sớm, giăng rằm:
Chỉ thanh đạm thế, âm thầm thế thôi.
Tôi về đây, đã lâu rồi,
Nằm trong cô tịch nhớ người phồn hoa
Tóc tơ, mình liễu da ngà,
Một người càng nhớ, càng xa một người
Ngày trông mây trắng bay hoài,
Đêm mơ áo trắng bay dài năm canh
Lòng vàng lạc cánh chim xanh,
Lạc từ cái ý chung tình lạc đi.
Chẳng điên chẳng dại là gì.
Bổng dưng mà biệt mà ly mọi người.
Chưa xa đã nhớ nhau rồi.
Nữa là hơn một tháng giời xa nhau.
Người đi nghỉ mát những đâu,
Đồ Sơn, Tam Đảo, 

In [12]:
def tokenize_function(example):
    tokenized = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_datasets = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
}

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    return_tensors="pt"
)

In [14]:
# Test batch after collate
batch = [tokenized_datasets["train"][i] for i in range(2)]
collated_batch = data_collator(batch)
print(collated_batch.keys())
print("Input IDs shape:", collated_batch["input_ids"].shape)
print("Labels shape:", collated_batch["labels"].shape)

dict_keys(['input_ids', 'attention_mask', 'labels'])
Input IDs shape: torch.Size([2, 2048])
Labels shape: torch.Size([2, 2048])


In [15]:
from transformers import TrainingArguments, EarlyStoppingCallback, TrainerCallback

class PrintLogCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(logs)
        return control

training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=70,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=5,
    save_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    seed=3407,
    report_to=[],
    logging_steps=1,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    max_seq_length=max_seq_length,
    data_collator=data_collator,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

Converting eval dataset to ChatML (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

Truncating eval dataset (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

In [16]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5))
trainer.add_callback(PrintLogCallback())

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 90 | Num Epochs = 7
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 70
 "-____-"     Number of trainable parameters = 12,156,928


Step,Training Loss,Validation Loss
5,7.858600,10.112890
10,6.402700,8.057552
15,5.595500,6.891245
20,5.777700,5.957647
25,5.345700,5.595487
30,5.567000,5.625077
35,4.863600,5.569949
40,5.466100,5.539271
45,5.300400,5.554359
50,5.205200,5.546415


{'loss': 10.2236, 'grad_norm': 17.038898468017578, 'learning_rate': 4e-05, 'epoch': 0.08888888888888889}
{'loss': 9.6301, 'grad_norm': 14.950401306152344, 'learning_rate': 8e-05, 'epoch': 0.17777777777777778}
{'loss': 8.9861, 'grad_norm': 14.602652549743652, 'learning_rate': 0.00012, 'epoch': 0.26666666666666666}
{'loss': 9.7655, 'grad_norm': 20.49504280090332, 'learning_rate': 0.00016, 'epoch': 0.35555555555555557}


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


{'loss': 7.8586, 'grad_norm': 9.982782363891602, 'learning_rate': 0.0002, 'epoch': 0.4444444444444444}
{'eval_loss': 10.112890243530273, 'eval_runtime': 17.8703, 'eval_samples_per_second': 0.56, 'eval_steps_per_second': 0.28, 'epoch': 0.4444444444444444}
{'loss': 6.9742, 'grad_norm': 9.110990524291992, 'learning_rate': 0.00019692307692307696, 'epoch': 0.5333333333333333}
{'loss': 6.5842, 'grad_norm': 4.630589008331299, 'learning_rate': 0.00019384615384615385, 'epoch': 0.6222222222222222}
{'loss': 7.2559, 'grad_norm': 4.542369365692139, 'learning_rate': 0.0001907692307692308, 'epoch': 0.7111111111111111}
{'loss': 6.7651, 'grad_norm': 2.8750319480895996, 'learning_rate': 0.0001876923076923077, 'epoch': 0.8}
{'loss': 6.4027, 'grad_norm': 2.3458938598632812, 'learning_rate': 0.00018461538461538463, 'epoch': 0.8888888888888888}
{'eval_loss': 8.057552337646484, 'eval_runtime': 16.7832, 'eval_samples_per_second': 0.596, 'eval_steps_per_second': 0.298, 'epoch': 0.8888888888888888}
{'loss': 6.4

TrainOutput(global_step=70, training_loss=5.605535166604178, metrics={'train_runtime': 1678.9523, 'train_samples_per_second': 0.334, 'train_steps_per_second': 0.042, 'total_flos': 1.84366607106048e+16, 'train_loss': 5.605535166604178})

In [17]:
save_path = "poemlucbat_gen_finetuned_llama3.2"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=save_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

In [18]:
def generate_poem(prompt, max_new_tokens=200):
    system_prompt = textwrap.dedent("""\
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        Bạn là một AI có khả năng sáng tác thơ lục bát, và phải tuân thủ nghiêm ngặt quy tắc của thơ lục bát:
            - Bắt đầu với dòng thơ 6 chữ, tiếp theo là dòng thơ 8 chữ và lặp lại.
            - Dòng thơ cuối cùng phải kết thúc bằng dòng thơ 8 chữ.
        <|eot_id|>
        """)

    user_prompt = textwrap.dedent(f"""\
        <|start_header_id|>user<|end_header_id|>
        Hãy sáng tác một bài thơ lục bát về chủ đề '{prompt}'.
        <|eot_id|>

        <|start_header_id|>assistant<|end_header_id|>
        """)

    input_text = system_prompt + "\n" + user_prompt

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_seq_length
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}

    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.8,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.1
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_poem = generated_text.replace(input_text, "").strip()
    return generated_poem

In [19]:
theme = "Mừng ngày quốc tế phụ nữ"
print(f"Chủ đề: {theme}\n")
print(generate_poem(theme))

Chủ đề: Mừng ngày quốc tế phụ nữ

system
Bạn là một AI có khả năng sáng tác thơ lục bát, và phải tuân thủ nghiêm ngặt quy tắc của thơ lục bát:
    - Bắt đầu với dòng thơ 6 chữ, tiếp theo là dòng thơ 8 chữ và lặp lại.
    - Dòng thơ cuối cùng phải kết thúc bằng dòng thơ 8 chữ.


user
Hãy sáng tác một bài thơ lục bát về chủ đề 'Mừng ngày quốc tế phụ nữ'.


assistant
Tốt đẹp hơn cả đời tôi,
Mất bao nhiêu, ta cũng nếm được nước hoa tươi ngon.
Hồi xưa mười tuổi,
Đường đi chơi vắng vẻ bỗng dặn dò đàng xa.
Vợ chồng, chén nọ, bàn nọ,
Một đời anh hùng không thiếu gì nhưng tình yêu thương mới thật sự.
Gặp nhau trong phố đông,
Nóng nàn vào mùa thu, lạnh lẽo vào mùa đông.
Khi một người đã có chồng,
Để con bớt gánh nặng, để cha mẹ vui lòng.
Ngôi nhà trở nên ấm cúng,
Quê hương đất nước thêm nhiều sắc màu.
Trong ngôi nhà, dưới cây sương,
Cũng đâu thôi, em có vợ chồng, cha mẹ rồi.
Câu chuyện ấy đã qua,
Dưới trần gian đã qua nhưng vẫn còn nhớ.
Sự tình ấy đã qua,
Dưới trần gian đã qua nhưng còn nhiều đi